# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

sum_value = 0

for sqrt_value in generator:
    sum_value += sqrt_value
    print(sqrt_value)

print(f"sum value is : {sum_value:.4}")

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
sum value is : 8.382


In [8]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(f"{sqrt_value:.5}")

1.0
1.4142
1.7321
2.0
2.2361
2.4495
2.6458
2.8284
3.0
3.1623
3.3166
3.4641
3.6056


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [1]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

In [6]:
# Question 3
import dlt
import duckdb

# create pipeline that load to duckdb
people_pipeline = dlt.pipeline(destination='duckdb', dataset_name='peoples_data')

# run the pipeline for first generators
info = people_pipeline.run(people_1(),
                           table_name='people',
                           write_disposition="replace",
                           primary_key='ID')

# show the outcome

conn = duckdb.connect(f"{people_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{people_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n people with only first generator:")
peoples = conn.sql("SELECT * FROM people").df()
display(peoples)

print("\n\n\n Sum All Age on table:")
sum_age = conn.sql("SELECT SUM(AGE) FROM people").df()
display(sum_age)

# append second generator to the table
info = people_pipeline.run(people_2(),
                           table_name='people',
                           write_disposition="append",
                           primary_key='ID')

print("\n\n\n people with both generator:")
all_peoples = conn.sql("SELECT * FROM people").df()
display(all_peoples)

print("\n\n\n Sum All Age on table:")
sum_all = conn.sql("SELECT SUM(AGE) FROM people").df()
display(sum_all)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people with only first generator:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708243099.6496708,YrnEJPxO+0492Q
1,2,Person_2,27,City_A,1708243099.6496708,WxhYDDgNiinbfQ
2,3,Person_3,28,City_A,1708243099.6496708,giSzYlPtmcp8MQ
3,4,Person_4,29,City_A,1708243099.6496708,y9F7bcJZJtIyAA
4,5,Person_5,30,City_A,1708243099.6496708,Xz/Owl5ivw8P8w





 Sum All Age on table:


,sum(AGE)
0,140.0





 people with both generator:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708243099.6496708,YrnEJPxO+0492Q,None
1,2,Person_2,27,City_A,1708243099.6496708,WxhYDDgNiinbfQ,None
2,3,Person_3,28,City_A,1708243099.6496708,giSzYlPtmcp8MQ,None
3,4,Person_4,29,City_A,1708243099.6496708,y9F7bcJZJtIyAA,None
4,5,Person_5,30,City_A,1708243099.6496708,Xz/Owl5ivw8P8w,None
5,3,Person_3,33,City_B,1708243100.338592,ttQKJliBNRn0fw,Job_3
6,4,Person_4,34,City_B,1708243100.338592,gT3Si94zaCzGaQ,Job_4
7,5,Person_5,35,City_B,1708243100.338592,oYZr9Cc41DfcRg,Job_5
8,6,Person_6,36,City_B,1708243100.338592,59gVA4KLxdEJFA,Job_6
9,7,Person_7,37,City_B,1708243100.338592,qSH7BLWxbnFk9Q,Job_7





 Sum All Age on table:


,sum(AGE)
0,353.0


In [7]:
# Question 4
import dlt
import duckdb

# create pipeline that load to duckdb
people_pipeline = dlt.pipeline(destination='duckdb', dataset_name='peoples_data')

# run the pipeline for first generators
info = people_pipeline.run(people_1(),
                           table_name='people',
                           write_disposition="replace",
                           primary_key='ID')

# merge second generator to the table
info = people_pipeline.run(people_2(),
                           table_name='people',
                           write_disposition="merge",
                           primary_key='ID')

# show the outcome

conn = duckdb.connect(f"{people_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{people_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n people with both generator:")
all_peoples = conn.sql("SELECT * FROM people").df()
display(all_peoples)

print("\n\n\n Sum All Age on table:")
sum_all = conn.sql("SELECT SUM(AGE) FROM people").df()
display(sum_all)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people with both generator:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708243643.0797598,Lls/gd9azB4dJg,None
1,2,Person_2,27,City_A,1708243643.0797598,6dMbO+A+ussHZw,None
2,5,Person_5,35,City_B,1708243643.7328453,znb1fDAlKExa/w,Job_5
3,7,Person_7,37,City_B,1708243643.7328453,5iONKrRrLpRyqw,Job_7
4,8,Person_8,38,City_B,1708243643.7328453,Su80hvfzvHFzdw,Job_8
5,4,Person_4,34,City_B,1708243643.7328453,0Lt+TC7M/Cwhug,Job_4
6,3,Person_3,33,City_B,1708243643.7328453,Z1zdqCARbnak4w,Job_3
7,6,Person_6,36,City_B,1708243643.7328453,I8EOSr81jg36Tw,Job_6





 Sum All Age on table:


,sum(AGE)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX